## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-01-14-52-05 +0000,nypost,Teacher accused in Devil’s Den murders of marr...,https://nypost.com/2025/08/01/us-news/teacher-...
1,2025-08-01-14-45-50 +0000,startribune,Minneapolis Indigenous groups seek police help...,https://www.startribune.com/minneapolis-indian...
2,2025-08-01-14-45-17 +0000,nyt,"What J.M.W. Turner Saw, and What We Still Can See",https://www.nytimes.com/2025/08/01/arts/jmw-tu...
3,2025-08-01-14-40-06 +0000,startribune,US employers slash hiring as Trump advances a ...,https://www.startribune.com/economy-likely-cre...
4,2025-08-01-14-36-29 +0000,nypost,Trump urges Fed board to ‘assume control’ if J...,https://nypost.com/2025/08/01/business/trump-u...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,trump,93
103,tariffs,32
102,new,23
112,tariff,21
388,president,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
204,2025-07-31-23-00-10 +0000,nypost,Trump struck huge trade deals — but tariffs ar...,https://nypost.com/2025/07/31/us-news/dozens-o...,212
318,2025-07-31-16-29-38 +0000,latimes,New Trump tariffs on Mexico delayed 90 days as...,https://www.latimes.com/world-nation/story/202...,181
164,2025-08-01-02-34-00 +0000,wsj,Lawmakers have introduced bills targeting Chin...,https://www.wsj.com/politics/policy/congress-l...,171
10,2025-08-01-14-34-26 +0000,nyt,Live Updates: Markets Fall on Weak Jobs Data a...,https://www.nytimes.com/live/2025/07/31/busine...,168
31,2025-08-01-13-51-03 +0000,wapo,Trump’s new tariffs will hit these major tradi...,https://www.washingtonpost.com/business/2025/0...,167


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
204,212,2025-07-31-23-00-10 +0000,nypost,Trump struck huge trade deals — but tariffs ar...,https://nypost.com/2025/07/31/us-news/dozens-o...
193,76,2025-07-31-23-44-33 +0000,nypost,White House releases new tariff rates for doze...,https://nypost.com/2025/07/31/us-news/white-ho...
182,72,2025-08-01-00-49-00 +0000,wsj,President Trump says he has averted a half-doz...,https://www.wsj.com/politics/policy/trumps-vow...
25,51,2025-08-01-14-08-00 +0000,wsj,"U.S. Envoy Steve Witkoff Visits Gaza as Trump,...",https://www.wsj.com/world/middle-east/u-s-envo...
22,48,2025-08-01-14-09-33 +0000,nypost,Devil’s Den suspect charged with killing coupl...,https://nypost.com/2025/08/01/us-news/devils-d...
134,48,2025-08-01-05-44-24 +0000,nypost,Kamala Harris complains ‘the system’ is ‘broke...,https://nypost.com/2025/08/01/us-news/kamala-h...
164,48,2025-08-01-02-34-00 +0000,wsj,Lawmakers have introduced bills targeting Chin...,https://www.wsj.com/politics/policy/congress-l...
337,46,2025-07-31-15-31-00 +0000,wsj,President Trump’s tighter deadline for Russia ...,https://www.wsj.com/politics/policy/trumps-rus...
67,39,2025-08-01-11-25-01 +0000,nypost,Russian attack on Kyiv leaves at least 28 dead...,https://nypost.com/2025/08/01/world-news/russi...
144,35,2025-08-01-04-42-31 +0000,nypost,Palestinian Islamic Jihad releases video of fr...,https://nypost.com/2025/08/01/world-news/pales...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
